In [2]:
import {
  ChatPromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts"
import { RunnableSequence } from "@langchain/core/runnables"
import { ChatOpenAI } from "@langchain/openai"
import { StringOutputParser } from "@langchain/core/output_parsers"
import { HumanMessage, AIMessage } from "@langchain/core/messages"

In [3]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  openAIApiKey: process.env.Tongyi_API_KEY,
  temperature: 1.5,
  modelName: "deepseek-v3",
  configuration: {
    baseURL: process.env.BASE_URL,
  },
}

## llm 改写提问

在自然的对话中用户经常会使用代词表示之前会话中提到过的事务，而这样是不利于向量检索的，因此我们可以使用 llm 重写用户的问题，使他们更加具体。

In [4]:
const rephraseChainPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "给定以下对话和一个后续问题，请将后续问题重述为一个独立的问题。请注意，重述的问题应该包含足够的信息，使得没有看过对话历史的人也能理解。",
  ],
  new MessagesPlaceholder("history"),
  ["human", "将以下问题重述为一个独立的问题：\n{question}"],
])


In [5]:
const rephraseChain = RunnableSequence.from([
  rephraseChainPrompt,
  new ChatOpenAI({
    ...chatOptions,
    temperature: 0.2,
  }),
  new StringOutputParser(),
])

In [6]:
const historyMessages = [
  new HumanMessage("你好， 我叫小明"),
  new AIMessage("你好，小明"),
]

const question = "你觉得我的名字怎么样？"

const standaloneQuestion = await rephraseChain.invoke({
  history: historyMessages,
  question,
})


In [7]:
console.log('standaloneQuestion', standaloneQuestion)

standaloneQuestion 你觉得“小明”这个名字怎么样？
